In [1]:
import pandas as pd
from math import sqrt, log
import sys
import time
import json



def _is_sym(seq):
    """Returns True if s is symmetric (same as rev. complement)"""
    comp = {
        'A': 'T',
        'T': 'A',
        'G': 'C',
        'C': 'G'
    }
    return seq == ''.join([comp[i] for i in seq][::-1])


def _overcount(st, p):
    ocu = 0
    x = 0
    while True:
        try:
            i = st.index(p, x)
        except ValueError:
            break
        ocu += 1
        x = i + 1
    return ocu


def _tercorr(st):
    _dh = 0
    _ds = -1.4 if _is_sym(st) else 0
    start = st[0]
    end = st[-1]

    if start == 'G' or start == 'C':
        _dh += 0.1
        _ds -= 2.8
    elif start == 'A' or start == 'T':
        _dh += 2.3
        _ds += 4.1

    if end == 'G' or end == 'C':
        _dh += 0.1
        _ds -= 2.8
    elif end == 'A' or end == 'T':
        _dh += 2.3
        _ds += 4.1
    return _dh, _ds

def temp(s, DNA_c=5000.0, Na_c=10.0, Mg_c=20.0, dNTPs_c=10.0, uncorrected=False):
    '''
    Returns the DNA/DNA melting temp using nearest-neighbor thermodynamics.

    This function returns better results than EMBOSS DAN because it uses updated
    thermodynamics values and takes into account initialization parameters from
    the work of SantaLucia (1998).

    Corrects for mono- and divalent cation concentrations.

    Arguments:
    - DNA_c:   DNA concentration [nM]
    - Na_c:    Na+ concentration [mM]
    - Mg_c:    Mg2+ concentration [mM]
    - dNTPs_c: dNTP concentration [mM]
    - correction: correct for cation concentration?
    '''

    R = 1.987    # Universal gas constant (cal/(K*mol))
    s = s.upper()
    dh, ds = _tercorr(s)
    k = DNA_c * 1e-9

    # Adapted from Table 1 in Allawi and SantaLucia (1997).
    # delta H (kcal/mol)
    dh_coeffs = {"AA": -7.9, "TT": -7.9,
                 "AT": -7.2,
                 "TA": -7.2,
                 "CA": -8.5, "TG": -8.5,
                 "GT": -8.4, "AC": -8.4,
                 "CT": -7.8, "AG": -7.8,
                 "GA": -8.2, "TC": -8.2,
                 "CG": -10.6,
                 "GC": -9.8,
                 "GG": -8.0, "CC": -8.0}

    # delta S (eu)
    ds_coeffs = {"AA": -22.2, "TT": -22.2,
                 "AT": -20.4,
                 "TA": -21.3,
                 "CA": -22.7, "TG": -22.7,
                 "GT": -22.4, "AC": -22.4,
                 "CT": -21.0, "AG": -21.0,
                 "GA": -22.2, "TC": -22.2,
                 "CG": -27.2,
                 "GC": -24.4,
                 "GG": -19.9, "CC": -19.9}

    # Multiplies the number of times each nuc pair is in the sequence by the
    # appropriate coefficient, then returns the sum of all the pairs
    dh = dh + \
        sum(_overcount(s, pair) * coeff for pair, coeff in dh_coeffs.items())
    ds = ds + \
        sum(_overcount(s, pair) * coeff for pair, coeff in ds_coeffs.items())

    fgc = len([filter(lambda x: x == 'G' or x == 'C', s)]) / float(len(s))

    # Melting temperature
    tm = (1000 * dh) / (ds + (R * log(k)))

    if uncorrected:
        return tm - 273.15

    MNa = Na_c * 1e-3
    MMg = Mg_c * 1e-3
    MdNTPs = dNTPs_c * 1e-3

    # Free magnesium concentration
    Ka = 3e4  # association constant in biological buffers
    D = (Ka * MdNTPs - Ka * MMg + 1)**2 + (4 * Ka * MMg)
    Fmg = (-(Ka * MdNTPs - Ka * MMg + 1) + sqrt(D)) / (2 * Ka)

    cation_ratio = sqrt(Fmg) / MNa if MNa > 0 else 7.0

    if cation_ratio < 0.22:
        tm = 1 / (
            (1 / tm) +
            ((4.29 * fgc - 3.95) * log(MNa) + 0.94 * log(MNa)**2) * 1e-5)
    else:
        a = 3.92
        d = 1.42
        g = 8.31
        Fmg = MMg
        if cation_ratio < 6.0:
            a = a * (0.843 - 0.352 * sqrt(MNa) * log(MNa))
            d = d * \
                (1.279 - 4.03 * log(MNa) * 1e-3 - 8.03 * log(MNa)**2 * 1e-3)
            g = g * (0.486 - 0.258 * log(MNa) + 5.25 * log(MNa)**3 * 1e-3)
        tm = 1 / (
            (1 / tm) +
            (a - 0.911 * log(Fmg) + fgc * (6.26 + d * log(Fmg)) +
             1 / (2 * (len(s) - 1)) * (-48.2 + 52.5 * log(Fmg) +
                                       g * log(Fmg)**2)) * 1e-5)

    return tm - 273.15

# Algorithm for actually scoring the primer sets
def score_kmer(param_list, c1=0.1, c2=-5, c3=3, default_score=1):
    # FgOccur, BgOccur, degDiff, inEx
    # C1T + C2E + C3*R
    E = 0
    R = len(param_list[0]) / param_list[1]
    if(param_list[4] == True):
        E = -2
    
    score = c1*default_score + c2*E + c3*R
    return score

def kmers(foregrounds, background, exclusion_set, kmers=14, pref_min=10, pref_max_temp=120, min_temp=0, max_temp=150):
    
    value_dict = {}
    id_num = 0
    with open(background, 'r') as bg_file:
                bg_first_line = bg_file.readline()
                fg_data = bg_file.read().replace('\n', '')

                with open(exclusion_set, 'r') as ex_file:
                    bg_first_line = ex_file.readline()
                    ex_data = ex_file.read().replace('\n', '')
                    
                for foreground in foregrounds:

            #     filename = filepath + 'fg' + fg.split(' ')[0].strip('>') + '_' + 'bg' + bg.split(' ')[0].strip('>') + '.json'
                    file_id = 'foreground_' + str(id_num)
                    id_num += 1
                    with open(foreground, 'r') as fg_file:
                        fg_first_line = fg_file.readline()
                        bg_data = fg_file.read().replace('\n', '')

                        for i in range(int(len(bg_data)) - kmers):
                            cur_mer = bg_data[i:i+kmers]
                            inEx = False
                            defDiff = 0
                            if(cur_mer in fg_data):
                                cur_temp = temp(cur_mer)
                                if(min_temp <= cur_temp <= max_temp):
                                    fgOccur = fg_data.count(cur_mer)
                                    if(pref_min > cur_temp):
                                        defDiff = pref_min - cur_temp
                                    if(pref_max_temp < cur_temp):
                                        defDiff = pref_max_temp - cur_temp
                                    if(cur_mer in ex_data):
                                        inEx = True
                                    if(cur_mer in value_dict):
                                        value_dict[cur_mer][0].append(i)
                                        value_dict[cur_mer][2] += 1
            #                             value_dict[cur_mer]['data'][0]['Location'].append(i)
            #                             value_dict[cur_mer]['data'][2]['FgOccur'] += 1
                                    else:
            #                             value_dict[cur_mer] = {"data": [{"Location" : [i] }, {"Melting Temp": cur_temp}, {"FgOccur": 1}] }
                                        value_dict[file_id + str(i)] = [[i], cur_temp, fgOccur, defDiff, inEx, 0]
    for key in value_dict.keys():
        current_mer = value_dict[key]
        current_mer[-1] = score_kmer(current_mer)
        
    df = pd.DataFrame.from_dict(value_dict, orient='index', columns=["Location(s)", "Melting Temp", "#Fg", "InPrefRange", "InEx", "Score"])
    return df, foreground, background, value_dict

def find_kmers(foregrounds, background, exclusion_set, filepath):
    identity = 0
    file_dict = {}
    file_lookup = {}
    i = 0
    for file in foregrounds:
        kmer_df, fg, bg, value_dict = kmers(file, background, exclusion_set, filepath)
        filename = filepath + 'fg' + fg.split(' ')[0].strip('>') + '_' + 'bg' + bg.split(' ')[0].strip('>') + '.json'
        file_id = 'foreground_' + str(i)
        file_dict[file_id] = value_dict
        file_lookup[file_id] = filename
        i += 1
    return file_dict, file_lookup

def select_primer(json_file):
    with open('sequence_data.json') as json_data:
        json_data_files = json.load(json_data)
        df = pd.read_json('sequence_data.json')
        F = len(df['foreground_0'][1]['data'][0]['Location'])
        B = value_dict[cur_mer]['data'][2]['FgOccur']
    return df
    
        

In [2]:
bg = "/Users/thatcher/Documents/Graduate School/FofanovResearch/INF685/data/GCF_000007465.2_ASM746v2_genomic.fna"
ex = "/Users/thatcher/Documents/Graduate School/FofanovResearch/INF685/data/exclude.fasta"
fgs = ["/Users/thatcher/Documents/Graduate School/FofanovResearch/INF685/data/Chromosone16.fasta", "/Users/thatcher/Documents/Graduate School/FofanovResearch/INF685/data/Chromosone17.fasta"]
start = time.process_time()
kmer_df, fg, bg, value_dict = kmers(fgs, bg, ex)
end = time.process_time()
print("Time Taken: {}".format(end - start))

Time Taken: 970.718661


In [3]:
# df = select_primer('sequence_data.json')
# len(df['foreground_0'][1]['data'][0]['Location'])
kmer_df

Location(s)  Melting Temp  #Fg  InPrefRange   InEx     Score
foreground_0397         [397]     62.327110    1            0  False  0.148133
foreground_0442         [442]     66.233929    1            0  False  0.145294
foreground_0443         [443]     65.856277    1            0  False  0.145554
foreground_01202       [1202]     53.775107    1            0  False  0.155788
foreground_01270       [1270]     48.130340    1            0  False  0.162331
foreground_01283       [1283]     45.095755    1            0  False  0.166525
foreground_01284       [1284]     45.095755    1            0  False  0.166525
foreground_01292       [1292]     49.687484    1            0  False  0.160377
foreground_01380       [1380]     55.119370    1            0  False  0.154427
foreground_01457       [1457]     53.658674    1            0  False  0.155909
foreground_01458       [1458]     50.928754    1            0  False  0.158906
foreground_01463       [1463]     44.030364    1            0  False  0.168135
foreground_01466       [1466]     45.304430    1            0  False  0.166219
foreground_01507       [1507]     56.977618    1            0  False  0.152652
foreground_01525       [1525]     60.519983    1            0  False  0.149570
foreground_01532       [1532]     61.913409    1            0  False  0.148455
foreground_01541       [1541]     64.402957    1            0  False  0.146582
foreground_01542       [1542]     66.260900    3            0  False  0.145276
foreground_01543       [1543]     65.991957    3            0  False  0.145460
foreground_01600       [1600]     49.513770    1            0  False  0.160589
foreground_01627       [1627]     64.402957    1            0  False  0.146582
foreground_01628       [1628]     66.260900    3            0  False  0.145276
foreground_01629       [1629]     65.991957    3            0  False  0.145460
foreground_01939       [1939]     59.193485    1            0  False  0.150681
foreground_02017       [2017]     71.854149    1            0  False  0.141751
foreground_02114       [2114]     68.389955    1            0  False  0.143866
foreground_02816       [2816]     65.914395    1            0  False  0.145514
foreground_03220       [3220]     71.549208    1            0  False  0.141929
foreground_03480       [3480]     66.393158    1            0  False  0.145185
foreground_03481       [3481]     69.201016    1            0  False  0.143352
...                       ...           ...  ...          ...    ...       ...
foreground_170866     [70866]     46.785196    1            0  False  0.164123
foreground_170867     [70867]     45.747424    2            0  False  0.165577
foreground_170979     [70979]     55.611428    1            0  False  0.153946
foreground_171335     [71335]     56.656908    1            0  False  0.152950
foreground_171338     [71338]     55.142321    1            0  False  0.154405
foreground_171355     [71355]     56.850888    1            0  False  0.152770
foreground_171356     [71356]     56.850888    2            0  False  0.152770
foreground_171357     [71357]     55.568065    1            0  False  0.153988
foreground_171358     [71358]     52.802728    1            0  False  0.156815
foreground_171369     [71369]     58.945562    1            0  False  0.150894
foreground_171370     [71370]     59.562579    1            0  False  0.150367
foreground_172364     [72364]     67.810705    1            0  False  0.144241
foreground_172679     [72679]     70.389348    1            0  False  0.142620
foreground_172680     [72680]     68.714511    1            0  False  0.143659
foreground_173404     [73404]     65.880490    1            0  False  0.145537
foreground_173638     [73638]     46.838488    1            0  False  0.164050
foreground_173810     [73810]     55.530987    1            0  False  0.154024
foreground_173832     [73832]     63.995347    1            0  False  0.146878
foreground_173836     [73836]     66.814698    1            0  False

In [7]:
kmer_df[kmer_df['InEx'] == True]

Empty DataFrame
Columns: [Location(s), Melting Temp, #Fg, InPrefRange, InEx, Score]
Index: []

In [5]:
kmer_df.to_json('primer_data.json', orient='index')
with open("primer_data.json", "r+") as json_data_file:
    data = json.load(json_data_file)
    json.dump(data, json_data_file, indent=4, sort_keys=True)
    

In [9]:
num_primer_sets = 10
kmer_df.nlargest(num_primer_sets, 'Score')

Location(s)  Melting Temp  #Fg  InPrefRange   InEx    Score
foreground_029891     [29891]      39.00675    1            0  False  0.17691
foreground_029894     [29894]      39.00675    1            0  False  0.17691
foreground_029897     [29897]      39.00675    1            0  False  0.17691
foreground_030624     [30624]      39.00675    1            0  False  0.17691
foreground_030924     [30924]      39.00675    1            0  False  0.17691
foreground_030927     [30927]      39.00675    1            0  False  0.17691
foreground_030940     [30940]      39.00675    1            0  False  0.17691
foreground_030943     [30943]      39.00675    1            0  False  0.17691
foreground_030946     [30946]      39.00675    1            0  False  0.17691
foreground_030949     [30949]      39.00675    1            0  False  0.17691